In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium.webdriver import Chrome
import json
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

In [2]:
import sys
print(sys.version)

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


In [3]:
start_url  = 'http://www.youtube.com'
delay=3
browser = Chrome()
browser.implicitly_wait(delay)

browser.get(start_url)  
browser.maximize_window()

In [4]:
input_ = browser.find_element_by_xpath('//*[@id="search"]')
input_

<selenium.webdriver.remote.webelement.WebElement (session="72968f3de49cfa1f802fd013afb7196a", element="4e4d7e99-af53-43e2-8c44-caaf61aa0c39")>

In [5]:
input_.send_keys('뜨뜨뜨뜨')

In [6]:
btn = browser.find_element_by_xpath('//*[@id="search-icon-legacy"]')
btn.click() 

In [7]:

body = browser.find_element_by_tag_name('body')#스크롤하기 위해 소스 추출

num_of_pagedowns = 4
#10번 밑으로 내리는 것
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1

In [7]:
html0 = browser.page_source

In [8]:
html = BeautifulSoup(html0,'html.parser')

In [9]:
video_list=html.find_all('a',class_='yt-simple-endpoint style-scope ytd-grid-video-renderer')

In [10]:
tester_url = []

for i in video_list:
    text = str(i)
    msg = re.search(r'href="(.*)', text, re.DOTALL)
    msg = start_url + msg.group(1)[:20]
    tester_url.append(msg)

In [11]:
tester_url = tester_url[:10]

In [12]:
len(tester_url)

10

In [13]:
tester_url

['http://www.youtube.com/watch?v=FLt7bCtsvNk',
 'http://www.youtube.com/watch?v=DpaAf7KRaZM',
 'http://www.youtube.com/watch?v=YOketbttgfg',
 'http://www.youtube.com/watch?v=10QkOikYI68',
 'http://www.youtube.com/watch?v=7XIfVxBNTXw',
 'http://www.youtube.com/watch?v=dszhmazAtSk',
 'http://www.youtube.com/watch?v=S5WN7gTYnfA',
 'http://www.youtube.com/watch?v=6Ef6ZNJmDLE',
 'http://www.youtube.com/watch?v=k3BqlalhjN4',
 'http://www.youtube.com/watch?v=h6M7Et0a2fY']

In [14]:
browser.get(tester_url[2])

In [15]:
body = browser.find_element_by_tag_name('body')#스크롤하기 위해 소스 추출

num_of_pagedowns = 2
#10번 밑으로 내리는 것
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(3)
    num_of_pagedowns -= 1
    
time.sleep(3)

In [16]:
soup0 = browser.page_source

In [17]:
soup = BeautifulSoup(soup0,'html.parser')

In [18]:
info1 = soup.find('div',{'id':'info-contents'})

In [19]:
comment = soup.find('yt-formatted-string',{'class':"count-text style-scope ytd-comments-header-renderer"}).text
title = info1.find('h1',{'class':'title style-scope ytd-video-primary-info-renderer'}).text
view =info1.find('ytd-video-view-count-renderer',{'class':'style-scope ytd-video-primary-info-renderer'}).find_all('span')[0].text
like = info1.find('div',{'id':'top-level-buttons'}).find_all('yt-formatted-string')[0].text
unlike = info1.find('div',{'id':'top-level-buttons'}).find_all('yt-formatted-string')[1].text
date = soup.find('div',{'id':'date'}).find('yt-formatted-string').text

In [20]:
comment

'댓글 2,509개'

In [21]:
title

'[배틀그라운드] 뜨뜨뜨뜨(DDDD) - 『솔큐』 하늘에서 보급 상자가 계속 떨어집니다. 블루홀 운영자님이 주시는 선물인가요!?'

In [22]:
view

'조회수 3,428,874회'

In [23]:
like

'1.9만'

In [24]:
unlike

'1.4천'

In [25]:
date

'2017. 11. 16.'

In [26]:
video_info = pd.DataFrame({'title':[],
                          'view':[],
                          'like':[],
                          'unlike':[],
                          'comment':[],
                          'date':[]})

In [27]:
for i in range(len(tester_url)):
    browser.get(tester_url[i])
    time.sleep(2)
    
    body = browser.find_element_by_tag_name('body')#스크롤하기 위해 소스 추출

    num_of_pagedowns = 2
    #10번 밑으로 내리는 것
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        num_of_pagedowns -= 1

    time.sleep(2)
    
    soup0 = browser.page_source
    time.sleep(3)
    soup = BeautifulSoup(soup0,'html.parser')
    info1 = soup.find('div',{'id':'info-contents'})
    try:
        comment = soup.find('yt-formatted-string',{'class':"count-text style-scope ytd-comments-header-renderer"}).text
    except:
        comment = '댓글x'
        
    title = info1.find('h1',{'class':'title style-scope ytd-video-primary-info-renderer'}).text
    view =info1.find('ytd-video-view-count-renderer',{'class':'style-scope ytd-video-primary-info-renderer'}).find_all('span')[0].text
    like = info1.find('div',{'id':'top-level-buttons'}).find_all('yt-formatted-string')[0].text
    unlike = info1.find('div',{'id':'top-level-buttons'}).find_all('yt-formatted-string')[1].text
    date = soup.find('div',{'id':'date'}).find('yt-formatted-string').text
    
    insert_data = pd.DataFrame({'title':[title],
                          'view':[view],
                          'like':[like],
                          'unlike':[unlike],
                          'comment':[comment],
                          'date':[date]})
    
    video_info = video_info.append(insert_data)
    
video_info.index = range(len(video_info))

In [28]:
video_info.index = range(len(video_info))

In [29]:
video_info

,title,view,like,unlike,comment,date
0,[배틀그라운드] 뜨뜨뜨뜨(DDDD) - 『솔쿼드 35킬 우승』 최다킬 기록 갱신! ...,"조회수 4,532,016회",2.5만,2.4천,"댓글 2,645개",2018. 3. 21.
1,"아프리카 멸망전,대회 출전 비제이 핵사용으로 인한 영정사건 [배틀그라운드]","조회수 3,502,641회",2.1만,1천,"댓글 6,894개",2019. 12. 18.
2,[배틀그라운드] 뜨뜨뜨뜨(DDDD) - 『솔큐』 하늘에서 보급 상자가 계속 떨어집니...,"조회수 3,428,876회",1.9만,1.4천,"댓글 2,509개",2017. 11. 16.
3,"단언컨대, 올타임 레전드 게임 [배틀그라운드]","조회수 3,289,626회",2.8만,1.4천,"댓글 2,029개",2019. 8. 1.
4,"트위치코리아의 갑질과, 내부고발자의 폭로로 드러난 영구정지 사건의 소름돋는 내막","조회수 2,862,139회",6.4만,2천,"댓글 9,163개",최초 공개: 2020. 4. 3.
5,배그 대회에서 역대급 캐리를 2번이나?? (지스타 배그 멸망전) - [배틀그라운드],"조회수 2,713,849회",1.9만,562,"댓글 2,787개",2018. 11. 16.
6,트위치 코리아의 부패와 영구정지 사태에 대한 진실,"조회수 2,711,401회",6.4만,2.2천,"댓글 7,318개",최초 공개: 2019. 5. 25.
7,치킨 먹으면 32인치 모니터 주는 미친 피시방 찾아가봤습니다 [배틀그라운드],"조회수 2,305,767회",2.4만,562,"댓글 3,163개",2020. 1. 21.
8,[배틀그라운드] 뜨뜨뜨뜨(QUAD) - 솔로 15킬 우승! 친구야 길리슈트 고맙다!...,"조회수 2,258,011회",7.4천,628,댓글 813개,2017. 9. 12.
9,숨만셔도 1초에 500원 존버의 끝 보여드립니다 - [배틀그라운드],"조회수 2,179,654회",1.5만,887,댓글 572개,2019. 5. 18.


In [30]:
video_info.to_csv('뜨뜨뜨뜨_#2.csv',index=False,encoding='cp949')

In [31]:
ttt = pd.read_csv('뜨뜨뜨뜨_#2.csv',engine='python',encoding='cp949')

In [32]:
len(ttt)

10

In [33]:
ttt['like'].iloc[1]

'2.1만'

In [34]:
#과연 유튜브 조회수는 게시한지 시간이 지날수록 즉, 올린지 꽤 된 영상의 조회수가 높을까?

In [35]:
like_ls = []
view_ls = []
unlike_ls = []
comment_ls = []
date_ls=[]
for i in range(len(ttt)):
    if '천' in ttt['like'].iloc[i]:
        a = ''.join(re.findall('[0-9]',ttt['like'].iloc[i]))
        if len(a) == 2:
            b = a+'00'
        else:
            b = a+'000'
    elif '만' in ttt['like'].iloc[i]:
        b = ''.join(re.findall('[0-9]',ttt['like'].iloc[i])) + '000'
    else:
        b = ttt['like'].iloc[i]
    like_ls.append(b)
    
    if '천' in ttt['unlike'].iloc[i]:
        aa = ''.join(re.findall('[0-9]',ttt['unlike'].iloc[i]))
        if len(aa) == 2:
            bb = aa+'00'
        else:
            bb = aa+'000'
    elif '만' in ttt['unlike'].iloc[i]:
        bb = ''.join(re.findall('[0-9]',ttt['unlike'].iloc[i])) + '000'
    else:
        bb = ttt['unlike'].iloc[i]
    unlike_ls.append(bb)
    
    view0 = ''.join(re.findall('[0-9]',ttt['view'].iloc[i]))
    view_ls.append(view0)
    
    comment0 = ''.join(re.findall('[0-9]',ttt['comment'].iloc[i]))
    comment_ls.append(comment0)
    
    date0 = ''.join(re.findall('[.0-9]',ttt['date'].iloc[i]))
    date_ls.append(date0[:-1])
    
ttt['like'] = like_ls
ttt['view'] = view_ls
ttt['comment'] = comment_ls
ttt['date']=date_ls
ttt['unlike'] = unlike_ls

In [36]:
ttt

,title,view,like,unlike,comment,date
0,[배틀그라운드] 뜨뜨뜨뜨(DDDD) - 『솔쿼드 35킬 우승』 최다킬 기록 갱신! ...,4532016,25000,2400,2645,2018.3.21
1,"아프리카 멸망전,대회 출전 비제이 핵사용으로 인한 영정사건 [배틀그라운드]",3502641,21000,1000,6894,2019.12.18
2,[배틀그라운드] 뜨뜨뜨뜨(DDDD) - 『솔큐』 하늘에서 보급 상자가 계속 떨어집니...,3428876,19000,1400,2509,2017.11.16
3,"단언컨대, 올타임 레전드 게임 [배틀그라운드]",3289626,28000,1400,2029,2019.8.1
4,"트위치코리아의 갑질과, 내부고발자의 폭로로 드러난 영구정지 사건의 소름돋는 내막",2862139,64000,2000,9163,2020.4.3
5,배그 대회에서 역대급 캐리를 2번이나?? (지스타 배그 멸망전) - [배틀그라운드],2713849,19000,562,2787,2018.11.16
6,트위치 코리아의 부패와 영구정지 사태에 대한 진실,2711401,64000,2200,7318,2019.5.25
7,치킨 먹으면 32인치 모니터 주는 미친 피시방 찾아가봤습니다 [배틀그라운드],2305767,24000,562,3163,2020.1.21
8,[배틀그라운드] 뜨뜨뜨뜨(QUAD) - 솔로 15킬 우승! 친구야 길리슈트 고맙다!...,2258011,7400,628,813,2017.9.12
9,숨만셔도 1초에 500원 존버의 끝 보여드립니다 - [배틀그라운드],2179654,15000,887,572,2019.5.18


In [37]:
ttt.to_csv('뜨뜨뜨뜨_#2.csv',encoding='cp949')

In [80]:
'''
가중치를 주기 위해

1. 일주일씩 끊어서 조회수의 합을 구한다음 평균을 내준다(일주일마다의 영상 올린 갯수가 다르므로)
2. 주별 영상 조회수 차이를 구한다 총 45주면 주별 조회수 차이는 44개가 나올 것
3. 나온 44개의 수치들의 평균을 구한다.
4. 계산을 통해서 주별 평균적인 조회수차이를 구할 수 있으므로 해당 수치로 날짜가 달라도 어느정도 감안하고 그 차이를 알 수 있다.
'''

'\n가중치를 주기 위해\n\n1. 일주일씩 끊어서 조회수의 합을 구한다음 평균을 내준다(일주일마다의 영상 올린 갯수가 다르므로)\n2. 주별 영상 조회수 차이를 구한다 총 45주면 주별 조회수 차이는 44개가 나올 것\n3. 나온 44개의 수치들의 평균을 구한다.\n4. 계산을 통해서 주별 평균적인 조회수차이를 구할 수 있으므로 해당 수치로 날짜가 달라도 어느정도 감안하고 그 차이를 알 수 있다.\n'

In [81]:
'''
날짜 별로 가중치를 주기 위해서
-> 오래 전에 업로드 된 영상일수록 조회수가 높을 가능성이 크기때문

1. 조회수의 평균을 먼저 구함
2. 평균 조회수까지 걸리는 시간을 구함
3. 그것을 걸린 날짜로 나눔
4. 그럼 해당 유튜버의 일자별 조회수 증가량을 알 수 있음 (max가 정해져 있음 but 영상이라는 것이 다른 외생변수도 많을테지만 일반화를 위해서 해당 방법 처럼 구함)

'''

'\n날짜 별로 가중치를 주기 위해서\n-> 오래 전에 업로드 된 영상일수록 조회수가 높을 가능성이 크기때문\n\n1. 조회수의 평균을 먼저 구함\n2. 평균 조회수까지 걸리는 시간을 구함\n3. 그것을 걸린 날짜로 나눔\n4. 그럼 해당 유튜버의 일자별 조회수 증가량을 알 수 있음 (max가 정해져 있음 but 영상이라는 것이 다른 외생변수도 많을테지만 일반화를 위해서 해당 방법 처럼 구함)\n\n'

In [82]:
ttt['view'] = ttt['view'].astype('float64')

In [83]:
ttt['view'].mean()

2583345.3